https://pypi.python.org/pypi/windrose/

https://github.com/python-windrose/windrose

http://nbviewer.jupyter.org/github/python-windrose/windrose/blob/master/windrose_sample_random.ipynb

In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import createFolder as cF
import datetime

import calc_date as cd
import netCDF4
import fill_values as fv
import save_fig as sF

from windrose_edit import WindroseAxes

In [ ]:
savefig = 0

stn = 'Haukeliseter' #
year= '2016'
month = '12'

hh = '00'

figdir = '../../Figures/Windrose/%s/MEPS' %(stn)
form = 'png'
cF.createFolder(figdir)      # for figure files
t = ['17', '18','19', '20', '21', '22', '23', '24', '25', '26', '27']
#t = ['26']

In [ ]:
level = '10m'
### MEPS
ncdir_10m = '../../Data/MEPS/%s/%s_%s' %(stn,level,hh)

### Haukeli obs
#txtdir = '../../Data/eklima/'
#txt_filename = 'Observations-hour_minutes.txt'

In [ ]:
champ = 255.
date_blue = np.array([1,74,159])/255.
no1 = np.array([79,94,26])/champ
no2 = np.array([131,156,45])/champ
no3 = np.array([71,153,112])/champ
no4 = np.array([77,111,157])/champ
no5 = np.array([157,58,55])/champ
no6 = np.array([211,120,50])/champ
no7 = np.array([218,181,70])/champ

In [ ]:
fn_10m = dict()

time_10m   = dict()
x_wind_10m = dict()
y_wind_10m = dict()

wd         = dict()
ws         = dict()

In [ ]:
for day in t:
    for ens_memb in range(0,10):
# read in the netcdf 10m file
        fn_10m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_10m,year,month,day,hh,ens_memb))
        time_10m[ens_memb] = fn_10m[ens_memb].variables['time']
        time_10m[ens_memb] = fv.fill_nan(time_10m[ens_memb][:])
        
        ini_day = (datetime.datetime.utcfromtimestamp(time_10m[0][0]).day)   # day of initialisation
        hour = (datetime.datetime.utcfromtimestamp(time_10m[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)

# Read in the variable name which should be plotted  
        x_wind_10m[ens_memb] = fn_10m[ens_memb].variables['x_wind_10m'][:]
        y_wind_10m[ens_memb] = fn_10m[ens_memb].variables['y_wind_10m'][:]
        
        x_wind_10m[ens_memb] = fv.fill_nan(x_wind_10m[ens_memb])
        y_wind_10m[ens_memb] = fv.fill_nan(y_wind_10m[ens_memb])
    
# Create wind speed and direction variables
        wd[ens_memb] = np.arctan2(y_wind_10m[ens_memb][:,0], x_wind_10m[ens_memb][:,0])
        ws[ens_memb] = np.sqrt((x_wind_10m[ens_memb][:,0])**2 + (y_wind_10m[ens_memb][:,0])**2) 

        wd[ens_memb] = 270-np.rad2deg(wd[ens_memb])  # convert from math to meteo direction and to degrees

        ws[ens_memb] = ws[ens_memb][~np.isnan(wd[ens_memb])]
        wd[ens_memb] = wd[ens_memb][~np.isnan(wd[ens_memb])]
#        if len(ws[ens_memb][:23]) == 0:
 #           continue
  #      else:
   #         print('%s.%s.%s' %(year,month,day),ens_memb,(ws[ens_memb][:23]).max())
    #### plot figure
    figname = '%s%s%s.%s' %(year,month,day,form)
    
    levels = np.arange(0,28,4)
    cmap = mpl.colors.ListedColormap([no1, no2, no3, no4, no5, no6, no7])
    norm = mpl.colors.BoundaryNorm(boundaries = levels, ncolors=cmap.N)
    plt.hist([0,1])
    plt.close()
    
    fig = plt.figure(figsize=(12,10))
    ax0 = fig.add_subplot(111, projection='windrose')
    ax0.bar(wd[0][:24], ws[0][:24], normed=True, opening=1.0, edgecolor='white',nsector=45, 
                 cmap=cmap,bins=np.arange(0, 28, 4))
    ax0._info['bins']
    ax0.set_legend()
    ax0.legend(title ='Wind$_{10m}$ [m s$^{-1}$]',fontsize=14)
    ax0.get_legend().get_title().set_fontsize('18')
    ax0.tick_params(labelsize = 20)
    ax0.set_title('%s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh), color = date_blue,fontsize = 20, y=1.08)
    

    if savefig == 1:
        sF.save_figure_portrait(figdir,figname,form)
        print('saved: %s/%s' %(figdir,figname) )
    else:
        plt.show()
    plt.close()
    
    for ens_memb in range(0,10):
        fn_10m[ens_memb].close()